In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import json
import os
import random
import glob
# from detectron_pro import detectron_mask_img,detectron_mask_img_composite
import shutil
import cv2

import random

In [2]:

# to get location that stickimg will sticked on jpg_dir center or random
def center_location(jpg_dir,stickimg_dir):
    im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    x_center,y_center = im.shape[1]/2,im.shape[0]/2
    im_stick = np.array(Image.open(stickimg_dir), dtype=np.uint8)
    im_stick_shape = im_stick.shape
    bd_box_x,bd_box_y = x_center-(im_stick_shape[1]/2),y_center-(im_stick_shape[0]/2)
    bd_box_length,bd_box_height =im_stick_shape[1],im_stick_shape[0]
    return bd_box_x,bd_box_y,bd_box_length,bd_box_height

def random_location(jpg_dir,stickimg_dir):
    im = np.array(Image.open(jpg_dir), dtype=np.uint8)
    # x boundary
    rangeX = 120
    x_left_bound,x_right_bound = rangeX, im.shape[1]-rangeX
    x_center = random.randrange(x_left_bound, x_right_bound)
    seq = [350,300,250,200]
    y_center = random.choice(seq)
    i_shape1 = [128,64]
    i_shape2 = [128*7/8,64*7/8]
    i_shape3 = [128*3/4,64*3/4]
    i_shape4 = [128*2/4,64*2/4]
    dict = {350:i_shape1,300:i_shape2 ,250:i_shape3,200:i_shape4}
    image_shape = dict[y_center]
    bd_box_x,bd_box_y = x_center-(image_shape[1]/2),y_center-(image_shape[0]/2)
    bd_box_length,bd_box_height =image_shape[1],image_shape[0]
    return bd_box_x,bd_box_y,bd_box_length,bd_box_height

In [6]:
#it can generate json file defined by your street_jpg and people_jpg 

def create_json_file(jpg_dir,street_json,stickimg_dir,results_dir,function='center'):
    if function == 'center':
        bd_box_x,bd_box_y,bd_box_length,bd_box_height = center_location(jpg_dir,stickimg_dir)
    elif function == 'random':
        bd_box_x,bd_box_y,bd_box_length,bd_box_height = random_location(jpg_dir,stickimg_dir)
        
    input_file = open (street_json)
    json_array = json.load(input_file)
    
    data = []
    data.append({
    'end':0,
    'hide':0,
    'id':0,
    'init':0,
    'lbl':"pasted_person",
    'lock':0,
    'occl':0,
    'pos':[
    bd_box_x,
    bd_box_y,
    bd_box_length,
    bd_box_height],    
    'posv':[
    0,
    0,
    0,
    0],
    'str':0
                })
    
    if json_array != []:
        for item in json_array:
            data.append(item)
            
    with open(results_dir, 'w') as outfile:
        json.dump(data, outfile)

# Start create

In [7]:
origin_data_dir = '/root/notebooks/final/caltech_conver_data'

In [8]:
img_data = glob.glob(origin_data_dir+'/**/*.jpg', recursive=True)

json_data = glob.glob(origin_data_dir+'/**/*.json', recursive=True)

In [9]:
img_data[:10]

[]

In [10]:
json_data[:10]

[]

In [13]:
# Image read dir
street_dir = '/root/notebooks/0858611-2/final_project/caltech_pedestrian_extractor/video_extractor/*'

people_dir = '/home/tedbest/final/market_mask2'

# Image save dir
save_dir = '/root/notebooks/final/result_dataset_3'

num_imgs = 20000

In [12]:
# Check dir folder exit
# If not, create one
if os.path.exists(save_dir) == False:
    os.makedirs(save_dir)

for s in ['people', 'mask', 'street', 'street_json','json']:
    if os.path.exists(os.path.join(save_dir, s)) == False:
        os.makedirs(os.path.join(save_dir, s))

PermissionError: [Errno 13] Permission denied: '/root/notebooks'

In [14]:
#street_imgs = glob.glob(street_dir+'/**/*.jpg', recursive=True)
street_imgs = img_data


#street_imgs = random.shuffle(random.sample(street_imgs, 5000))
street_imgs = random.sample(street_imgs, num_imgs)

random.shuffle(street_imgs)

ValueError: Sample larger than population or is negative

In [17]:
people_imgs_ = glob.glob(people_dir+'/people/*.jpg', recursive=True)

mask_imgs_ = [i.replace('people', 'mask') for i in people_imgs_]

#people_imgs = random.sample(people_imgs, num_imgs)

r_t_imgs = []
r_m_imgs = []

In [18]:
for i in range(len(people_imgs_)):
    if (i%500==0):
        print("Process (",i,"/",len(people_imgs_),")  ","{:.2f}".format(100*i/len(people_imgs_))," %")
    r_t_imgs.append(cv2.imread(people_imgs_[i]))
    r_m_imgs.append(cv2.imread(mask_imgs_[i]))

Process ( 0 / 61295 )   0.00  %
Process ( 500 / 61295 )   0.82  %
Process ( 1000 / 61295 )   1.63  %
Process ( 1500 / 61295 )   2.45  %
Process ( 2000 / 61295 )   3.26  %
Process ( 2500 / 61295 )   4.08  %
Process ( 3000 / 61295 )   4.89  %
Process ( 3500 / 61295 )   5.71  %
Process ( 4000 / 61295 )   6.53  %
Process ( 4500 / 61295 )   7.34  %
Process ( 5000 / 61295 )   8.16  %
Process ( 5500 / 61295 )   8.97  %
Process ( 6000 / 61295 )   9.79  %
Process ( 6500 / 61295 )   10.60  %
Process ( 7000 / 61295 )   11.42  %
Process ( 7500 / 61295 )   12.24  %
Process ( 8000 / 61295 )   13.05  %
Process ( 8500 / 61295 )   13.87  %
Process ( 9000 / 61295 )   14.68  %
Process ( 9500 / 61295 )   15.50  %
Process ( 10000 / 61295 )   16.31  %
Process ( 10500 / 61295 )   17.13  %
Process ( 11000 / 61295 )   17.95  %
Process ( 11500 / 61295 )   18.76  %
Process ( 12000 / 61295 )   19.58  %
Process ( 12500 / 61295 )   20.39  %
Process ( 13000 / 61295 )   21.21  %
Process ( 13500 / 61295 )   22.02  %
P

In [14]:
#people_imgs = [people_imgs_[random.randint(0,len(people_imgs_)-1)] for i in range(num_imgs)]


#random.shuffle(people_imgs)

In [15]:
for i in range(num_imgs):
    
    if (i%100==0):
        print("Process (",i,"/",num_imgs,")  ","{:.2f}".format(100*i/num_imgs)," %")
        
    people_pick = random.randint(0,len(r_m_imgs)-1)
    
    # create mask and save
    #try:
    #    mask_img = detectron_mask_img(people_imgs[i],(64,128))
    #    mask_img = Image.fromarray(mask_img)
    #except Exception as e:
    #    print("Skip image :",i)
    #    continue
    mask_img = r_m_imgs[people_pick]
        
    mask_img.save(save_dir+'/mask/'+str('{0:06}'.format(i))+'.jpg')
    
    # save street img
    street_img = cv2.imread(street_imgs[i])
    street_img = cv2.resize(street_img,(640,480))
    cv2.imwrite(save_dir+'/street/'+str('{0:06}'.format(i))+'.jpg', street_img)
    
    ################################################################
    img_path = street_imgs[i]
    json_dir = img_path.replace('images', 'annotations')
    json_dir = json_dir.replace('jpg', 'json')
    shutil.copyfile(json_dir, save_dir+'/street_json/'+str('{0:06}'.format(i))+'.json')
    ################################################################
    
    # save poeple img
    #people_img = cv2.imread(people_imgs[i])
    people_img = r_t_imgs[people_pick]
    people_img = cv2.resize(people_img,(64,128))
    cv2.imwrite(save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg', people_img)
    
    # create json file and save
    create_json_file(save_dir+'/street/'+str('{0:06}'.format(i))+'.jpg',
                     save_dir+'/street_json/'+str('{0:06}'.format(i))+'.json',
                     save_dir+'/people/'+str('{0:06}'.format(i))+'.jpg',
                     save_dir+'/json/'+str('{0:06}'.format(i))+'.json',
                     function="center")

Process ( 0 / 20000 )   0.00  %
Process ( 100 / 20000 )   0.50  %
Process ( 200 / 20000 )   1.00  %
Process ( 300 / 20000 )   1.50  %
Process ( 400 / 20000 )   2.00  %
Process ( 500 / 20000 )   2.50  %
Process ( 600 / 20000 )   3.00  %
Process ( 700 / 20000 )   3.50  %
Process ( 800 / 20000 )   4.00  %
Process ( 900 / 20000 )   4.50  %
Process ( 1000 / 20000 )   5.00  %
Process ( 1100 / 20000 )   5.50  %
Process ( 1200 / 20000 )   6.00  %
Process ( 1300 / 20000 )   6.50  %
Process ( 1400 / 20000 )   7.00  %
Process ( 1500 / 20000 )   7.50  %
Process ( 1600 / 20000 )   8.00  %
Process ( 1700 / 20000 )   8.50  %
Process ( 1800 / 20000 )   9.00  %
Process ( 1900 / 20000 )   9.50  %
Process ( 2000 / 20000 )   10.00  %
Process ( 2100 / 20000 )   10.50  %
Process ( 2200 / 20000 )   11.00  %
Process ( 2300 / 20000 )   11.50  %
Process ( 2400 / 20000 )   12.00  %
Process ( 2500 / 20000 )   12.50  %
Process ( 2600 / 20000 )   13.00  %
Process ( 2700 / 20000 )   13.50  %
Process ( 2800 / 20000 )